In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]       = Query[['a','z']]
    Src[['Symbol','TF']] = Query[['symbol','tf']]
    
    Src['Datetime']     = Query['datetime']
    Src['Date']         = Query['datetime'].dt.date
    Src['Time']         = Query['datetime'].dt.time
    
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc    = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Time+1']      = (Calc['Datetime'] + pd.Timedelta(hours=1)).dt.time

    Calc['Day Open']    = Calc_by['Open']  .transform('first')
    Calc['Day High']    = Calc_by['High']  .transform('max')
    Calc['Day Low']     = Calc_by['Low']   .transform('min')
    Calc['Day Close']   = Calc_by['Close'] .transform('last')
    Calc['Day Hilo']    = Calc['Day High'] - Calc['Day Low']

    Calc['ID High']     = Calc_by['High'] .expanding().max()    .reset_index(drop=1)
    Calc['ID Low']      = Calc_by['Low']  .expanding().min()    .reset_index(drop=1)
    Calc['ID Hilo']     = Calc['ID High'] - Calc['ID Low']

    Calc['Bwd HL']      = Calc['ID Hilo']

    Calc['Bwd Chg']     =                     Calc['Open'] - Calc['Day Open']
    Calc['Fwd Chg']     = Calc['Day Close'] - Calc['Open']

    Calc['Bwd Chg Abs'] = Calc['Bwd Chg'].abs()
    Calc['Fwd Chg Abs'] = Calc['Fwd Chg'].abs()


    Calc['Fwd Bull']    = Calc['Fwd Chg'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['Fwd Bear']    = Calc['Fwd Chg'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['Fwd Chg Pos'] = Calc['Fwd Chg Abs'] * Calc['Fwd Bull']
    Calc['Fwd Chg Neg'] = Calc['Fwd Chg Abs'] * Calc['Fwd Bear']

    Calc['Fwd HL']      = Calc['Day Hilo'] - Calc['ID Hilo']
    Calc['Fwd HL Pos']  = Calc['Fwd HL'] * Calc['Fwd Bull']
    Calc['Fwd HL Neg']  = Calc['Fwd HL'] * Calc['Fwd Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats 1

In [62]:
Calc.groupby('Time')[['Fwd Chg']].describe().round(0).astype(int)

Fwd Chg                                       
           count mean   std   min   25% 50%   75%   max
Time                                                   
09:00:00    1241  -26  1712 -7641 -1083  64  1015  5768
10:00:00    1241  -20  1597 -6425  -899  16   938  5420
11:00:00    1241  -41  1369 -7315  -696  32   720  5297
12:00:00    1241    0  1189 -6885  -614  18   650  5111
13:00:00    1246  -13  1071 -7834  -503  16   542  5224
14:00:00    1245  -17   955 -7018  -489  10   474  4585
15:00:00    1245  -14   889 -8005  -406   7   447  5697
16:00:00    1245   -4   708 -3739  -312  25   335  5544
17:00:00    1245   -8   478 -4214  -235  10   229  2005
18:00:00     676   19   285 -2530  -130  18   145  1121

In [63]:
Calc.groupby('Time')[['Fwd Chg Abs']].describe().round(0).astype(int)

Fwd Chg Abs                                       
               count  mean   std min  25%   50%   75%   max
Time                                                       
09:00:00        1241  1327  1081   0  505  1049  1849  7641
10:00:00        1241  1209  1043   0  423   914  1760  6425
11:00:00        1241  1005   930   0  332   712  1430  7315
12:00:00        1241   859   822   0  285   630  1192  6885
13:00:00        1246   752   762   0  234   520  1010  7834
14:00:00        1245   676   674   0  220   480   923  7018
15:00:00        1245   608   648   0  188   433   801  8005
16:00:00        1245   482   517   0  158   325   639  5544
17:00:00        1245   331   345   0  118   232   421  4214
18:00:00         676   197   207   0   66   140   267  2530

In [64]:
Calc.groupby('Time')[['Fwd Chg Pos']].describe().round(0).astype(int)

Fwd Chg Pos                                      
               count  mean   std min  25%  50%   75%   max
Time                                                      
09:00:00         642  1258  1022   0  452  980  1823  5768
10:00:00         632  1167   999   0  428  901  1648  5420
11:00:00         631   948   822   0  314  712  1353  5297
12:00:00         632   844   760   0  298  640  1222  5111
13:00:00         639   720   692   0  233  518   978  5224
14:00:00         628   654   630   0  228  469   901  4585
15:00:00         628   589   594   0  195  442   776  5697
16:00:00         643   463   510   0  158  318   610  5544
17:00:00         638   316   304   0  118  224   414  2005
18:00:00         365   201   193   0   68  134   297  1121

In [65]:
Calc.groupby('Time')[['Fwd Chg Neg']].describe().round(0).astype(int)

Fwd Chg Neg                                       
               count  mean   std   min   25%   50%  75% max
Time                                                       
09:00:00         599 -1402  1136 -7641 -1894 -1115 -578  -6
10:00:00         609 -1253  1086 -6425 -1811  -922 -415  -6
11:00:00         610 -1064  1027 -7315 -1549  -712 -354  -6
12:00:00         609  -875   882 -6885 -1175  -618 -258  -7
13:00:00         607  -785   828 -7834 -1046  -530 -236  -5
14:00:00         617  -699   716 -7018  -977  -496 -210  -5
15:00:00         617  -627   699 -8005  -836  -409 -180  -5
16:00:00         602  -503   525 -3739  -672  -329 -158  -5
17:00:00         607  -348   383 -4214  -450  -240 -118  -5
18:00:00         311  -194   222 -2530  -256  -142  -64  -5

In [66]:
# Calc.groupby('Time')[['Fwd HL']].describe().round(0).astype(int)

In [67]:
# Calc.groupby('Time')[['Fwd HL Pos']].describe().round(0).astype(int)

In [68]:
# Calc.groupby('Time')[['Fwd HL Neg']].describe().round(0).astype(int)

# Stats 2

In [69]:
def STATS(Calc, HR, X, Y):
    pipe = []
    for b in [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000]:

        Df = Calc[(Calc['Time'] == dt.time(HR,00)) & (Calc[X] > b)]

        pipe.append({
            'Time =': dt.time(HR,00),
            f'{X} >': b,
            **Df[Y].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)

In [70]:
STATS(Calc, HR=13, X='Bwd Chg Abs', Y='Fwd Chg Abs')

,Time =,Bwd Chg Abs >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,1169,752,767,0,235,520,1010,7834
1,13:00:00,200,1077,752,746,0,234,522,1012,5224
2,13:00:00,500,859,782,770,0,250,561,1043,5224
3,13:00:00,800,663,799,776,0,264,573,1071,5224
4,13:00:00,1000,534,840,797,0,272,611,1126,5224
5,13:00:00,1200,434,853,798,0,282,636,1124,5224
6,13:00:00,1500,297,922,858,5,289,715,1216,5224
7,13:00:00,1800,185,975,851,5,303,778,1319,4886
8,13:00:00,2000,139,1019,887,5,343,835,1378,4886
9,13:00:00,2200,109,1029,873,5,340,836,1498,4886


In [71]:
STATS(Calc, HR=13, X='Bwd Chg Abs', Y='Fwd HL')

,Time =,Bwd Chg Abs >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,1169,440,638,0,0,202,633,5824
1,13:00:00,200,1077,435,617,0,0,199,627,5012
2,13:00:00,500,859,437,626,0,0,207,622,5012
3,13:00:00,800,663,441,616,0,0,208,620,3895
4,13:00:00,1000,534,472,640,0,0,221,703,3778
5,13:00:00,1200,434,470,632,0,0,223,682,3778
6,13:00:00,1500,297,480,639,0,0,223,760,3778
7,13:00:00,1800,185,486,646,0,0,217,788,3182
8,13:00:00,2000,139,511,664,0,0,251,820,3182
9,13:00:00,2200,109,495,669,0,0,207,788,3182


In [72]:
STATS(Calc, HR=13, X='Bwd HL', Y='Fwd Chg Abs')

,Time =,Bwd HL >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,1246,752,762,0,234,520,1010,7834
1,13:00:00,200,1246,752,762,0,234,520,1010,7834
2,13:00:00,500,1246,752,762,0,234,520,1010,7834
3,13:00:00,800,1228,757,765,0,236,528,1019,7834
4,13:00:00,1000,1177,767,768,0,246,545,1034,7834
5,13:00:00,1200,1089,789,779,0,251,566,1070,7834
6,13:00:00,1500,925,835,815,0,273,593,1132,7834
7,13:00:00,1800,725,912,870,0,300,665,1225,7834
8,13:00:00,2000,602,939,900,0,295,712,1278,7834
9,13:00:00,2200,482,994,946,0,309,760,1359,7834


In [76]:
STATS(Calc, HR=13, X='Bwd HL', Y='Fwd HL')

,Time =,Bwd HL >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,1246,442,642,0,0,200,640,5824
1,13:00:00,200,1246,442,642,0,0,200,640,5824
2,13:00:00,500,1246,442,642,0,0,200,640,5824
3,13:00:00,800,1228,445,645,0,0,202,645,5824
4,13:00:00,1000,1177,447,647,0,0,203,648,5824
5,13:00:00,1200,1089,455,650,0,0,207,682,5824
6,13:00:00,1500,925,480,682,0,0,219,704,5824
7,13:00:00,1800,725,522,730,0,0,238,774,5824
8,13:00:00,2000,602,532,738,0,0,251,794,5824
9,13:00:00,2200,482,563,780,0,0,253,839,5824


# Snippets

In [74]:
def CALCULATIONS(Src):
    Calc    = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Time+1']      = (Calc['Datetime'] + pd.Timedelta(hours=1)).dt.time

    Calc['Day Open']    = Calc_by['Open']  .transform('first')
    Calc['Day High']    = Calc_by['High']  .transform('max')
    Calc['Day Low']     = Calc_by['Low']   .transform('min')
    Calc['Day Close']   = Calc_by['Close'] .transform('last')
    Calc['Day Hilo']    = Calc['Day High'] - Calc['Day Low']

    Calc['ID High']     = Calc_by['High'] .expanding().max()    .reset_index(drop=1)
    Calc['ID Low']      = Calc_by['Low']  .expanding().min()    .reset_index(drop=1)
    Calc['ID Hilo']     = Calc['ID High'] - Calc['ID Low']

    Calc['Bwd HL']      = Calc['ID Hilo']

    Calc['Bwd Chg']     =                     Calc['Open'] - Calc['Day Open']
    Calc['Fwd Chg']     = Calc['Day Close'] - Calc['Open']

    Calc['Bwd Chg Abs'] = Calc['Bwd Chg'].abs()
    Calc['Fwd Chg Abs'] = Calc['Fwd Chg'].abs()


    Calc['Fwd Bull']    = Calc['Fwd Chg'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['Fwd Bear']    = Calc['Fwd Chg'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['Fwd Chg Pos'] = Calc['Fwd Chg Abs'] * Calc['Fwd Bull']
    Calc['Fwd Chg Neg'] = Calc['Fwd Chg Abs'] * Calc['Fwd Bear']

    Calc['Fwd HL']      = Calc['Day Hilo'] - Calc['ID Hilo']
    Calc['Fwd HL Pos']  = Calc['Fwd HL'] * Calc['Fwd Bull']
    Calc['Fwd HL Neg']  = Calc['Fwd HL'] * Calc['Fwd Bear']
    return Calc

In [ ]:
def STATS(Calc, HR, X, Y):
    pipe = []
    for b in [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000]:

        Df = Calc[(Calc['Time'] == dt.time(HR,00)) & (Calc[X] > b)]

        pipe.append({
            'Time =': dt.time(HR,00),
            f'{X} >': b,
            **Df[Y].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)


Calc.groupby('Time')[['Fwd Chg']].describe().round(0).astype(int)
Calc.groupby('Time')[['Fwd Chg Abs']].describe().round(0).astype(int)
Calc.groupby('Time')[['Fwd Chg Pos']].describe().round(0).astype(int)
Calc.groupby('Time')[['Fwd Chg Neg']].describe().round(0).astype(int)